In [2]:
# !pip install transformers datasets scikit-learn pandas numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# ---------------------------------------------------------
# 1. 데이터 로드 및 전처리
# ---------------------------------------------------------
# encoding 이슈 대응: cp949로 먼저 시도하고 안되면 euc-kr
try:
    df = pd.read_csv('Hufs_Syllabus_Data.csv', encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv('Hufs_Syllabus_Data.csv', encoding='euc-kr')

# 결측치 처리
df['OtherNotes'] = df['OtherNotes'].fillna('')

# 모델 입력용 통합 텍스트 생성
df['FullText'] = (
    "과목명: " + df['CourseTitle'] + " / " +
    "수업개요: " + df['CourseDescription'] + " / " +
    "수업방식: " + df['TeachingMethods'] + " / " +
    "기타사항: " + df['OtherNotes']
)

# ---------------------------------------------------------
# 2. 라벨링 (Target Labels) - Rule-based 보정
# ---------------------------------------------------------
df['Is_Teamplay'] = (df['EtcScore'] > 0).astype(int)

def adjust_teamplay_label(row):
    text = row['FullText']
    keywords = ['조별', '팀', '프로젝트', '동료 평가', 'Team', 'Project']

    if row['Is_Teamplay'] == 1:
        return 1
    for kw in keywords:
        if kw in text:
            return 1
    return 0

df['Is_Teamplay'] = df.apply(adjust_teamplay_label, axis=1)

# 과제 부담 지수 라벨링 (0.0 ~ 1.0)
df['Burden_Score'] = (df['AssignmentScore'] + df['EtcScore']) / 100.0

print(f"전체 데이터 수: {len(df)}")
print(f"팀플(1) 비율: {df['Is_Teamplay'].mean():.2f}")

# ---------------------------------------------------------
# 3. 데이터 분할 (Stratified Split)
# ---------------------------------------------------------
# Train: 80%, Val: 10%, Test: 10%
train_val_df, test_df = train_test_split(
    df, test_size=0.1, random_state=42, stratify=df['Is_Teamplay']
)

train_df, val_df = train_test_split(
    train_val_df, test_size=0.11, random_state=42, stratify=train_val_df['Is_Teamplay']
)

# ---------------------------------------------------------
# 4. 분할된 데이터셋 저장 (NEW!)
# ---------------------------------------------------------
# 나중에 다시 돌려도 똑같은 데이터로 학습/평가하기 위해 저장합니다.
# encoding='utf-8-sig'는 엑셀에서 한글이 깨지지 않게 해줍니다.

train_df.to_csv('train.csv', index=False, encoding='utf-8-sig')
val_df.to_csv('val.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('test.csv', index=False, encoding='utf-8-sig')

print("-" * 30)
print("✅ 데이터 분할 및 저장 완료!")
print(f"Train Set: {len(train_df)}개 -> 'train.csv' 저장됨")
print(f"Valid Set: {len(val_df)}개 -> 'val.csv' 저장됨")
print(f"Test Set : {len(test_df)}개 -> 'test.csv' 저장됨")
print("-" * 30)

# 데이터 확인
train_df[['CourseTitle', 'Is_Teamplay']].head()


전체 데이터 수: 100
팀플(1) 비율: 0.61
------------------------------
✅ 데이터 분할 및 저장 완료!
Train Set: 80개 -> 'train.csv' 저장됨
Valid Set: 10개 -> 'val.csv' 저장됨
Test Set : 10개 -> 'test.csv' 저장됨
------------------------------


,CourseTitle,Is_Teamplay
79,시민의사회적책임,1
39,젠더와문학,0
77,글로벌창업,1
49,오페라의감상,0
81,한중일문학산책,1


In [4]:
# [training.ipynb] - Cell 2: Dataset Class & Model Training Loop (최종 수정)

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # PyTorch의 AdamW 사용
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm
import numpy as np
import os

# 1. Hyperparameters 설정
MODEL_NAME = "monologg/distilkobert"
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 5e-5

# 2. Custom Dataset 클래스 정의
class SyllabusDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['FullText'].to_numpy()
        self.labels = df['Is_Teamplay'].to_numpy() # 0 or 1
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터로더 생성 함수
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = SyllabusDataset(
        df=df,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0,
        shuffle=True
    )

# 4. 토크나이저 로드
print(f"{MODEL_NAME} 모델 로드 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

# 5. 모델 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model = model.to(device)

# 6. Optimizer
# [수정] correct_bias=False 옵션을 삭제했습니다.
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

# 7. 학습 함수
def train_epoch(model, data_loader, loss_fn, optimizer, device, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

# 8. 평가 함수
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

# --- 학습 실행 ---
print("🚀 학습 시작 (DistilKoBERT) 🚀")
best_accuracy = 0

for epoch in range(EPOCHS):
    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        len(train_df)
    )

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val_df)
    )

    print(f'Epoch {epoch + 1}/{EPOCHS} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'model.pt')
        best_accuracy = val_acc
        print(f"✅ Best model saved! (Acc: {best_accuracy:.4f})")

monologg/distilkobert 모델 로드 중...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

The repository monologg/distilkobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/distilkobert .
 You can inspect the repository content at https://hf.co/monologg/distilkobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


tokenization_kobert.py: 0.00B [00:00, ?B/s]

...

사용 디바이스: cuda


config.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 학습 시작 (DistilKoBERT) 🚀
Epoch 1/10 | Train Loss: 0.6857 Acc: 0.6000 | Val Loss: 0.6687 Acc: 0.6000
✅ Best model saved! (Acc: 0.6000)
Epoch 2/10 | Train Loss: 0.6474 Acc: 0.6125 | Val Loss: 0.6614 Acc: 0.6000
Epoch 3/10 | Train Loss: 0.6448 Acc: 0.6125 | Val Loss: 0.6553 Acc: 0.6000
Epoch 4/10 | Train Loss: 0.6146 Acc: 0.6125 | Val Loss: 0.6473 Acc: 0.6000
Epoch 5/10 | Train Loss: 0.6054 Acc: 0.6250 | Val Loss: 0.6357 Acc: 0.6000
Epoch 6/10 | Train Loss: 0.5630 Acc: 0.7500 | Val Loss: 0.6294 Acc: 0.6000
Epoch 7/10 | Train Loss: 0.5202 Acc: 0.7375 | Val Loss: 0.6309 Acc: 0.6000
Epoch 8/10 | Train Loss: 0.4582 Acc: 0.8375 | Val Loss: 0.6058 Acc: 0.7000
✅ Best model saved! (Acc: 0.7000)
Epoch 9/10 | Train Loss: 0.3781 Acc: 0.9750 | Val Loss: 0.6266 Acc: 0.6000
Epoch 10/10 | Train Loss: 0.2914 Acc: 0.9625 | Val Loss: 0.5676 Acc: 0.8000
✅ Best model saved! (Acc: 0.8000)


In [5]:
# [training.ipynb] - Cell 3: Regression Model Training (Burden Score)

# ---------------------------------------------------------
# 1. Regression용 데이터셋 클래스 정의
# ---------------------------------------------------------
class RegressionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['FullText'].to_numpy()
        # [중요] 회귀(Regression)는 라벨이 실수형(Float)이어야 합니다.
        self.labels = df['Burden_Score'].to_numpy()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float) # Float 형변환 필수
        }

# ---------------------------------------------------------
# 2. 데이터로더 준비
# ---------------------------------------------------------
def create_reg_loader(df, tokenizer, max_len, batch_size):
    ds = RegressionDataset(df, tokenizer, max_len)
    return DataLoader(ds, batch_size=batch_size, num_workers=0, shuffle=True)

# 기존에 로드한 tokenizer 사용
train_reg_loader = create_reg_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_reg_loader = create_reg_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)

# ---------------------------------------------------------
# 3. 모델 초기화 (Regression Mode)
# ---------------------------------------------------------
# num_labels=1로 설정하면 자동으로 Regression(MSELoss) 모드로 동작합니다.
burden_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)
burden_model = burden_model.to(device)

reg_optimizer = AdamW(burden_model.parameters(), lr=2e-5) # 학습률 살짝 낮춤
reg_loss_fn = torch.nn.MSELoss().to(device) # 평균 제곱 오차 (회귀용)

# ---------------------------------------------------------
# 4. 학습 루프 (Regression용)
# ---------------------------------------------------------
def train_reg_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss # Huggingface 모델은 내부적으로 Loss 계산됨

        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return np.mean(losses)

def eval_reg_model(model, data_loader, loss_fn, device):
    model = model.eval()
    losses = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            losses.append(loss.item())
    return np.mean(losses)

# ---------------------------------------------------------
# 5. 학습 실행
# ---------------------------------------------------------
print("🚀 과제 부담 예측 모델(Regression) 학습 시작 🚀")
best_loss = float('inf')

for epoch in range(10): # 10 Epoch
    train_loss = train_reg_epoch(burden_model, train_reg_loader, reg_loss_fn, reg_optimizer, device)
    val_loss = eval_reg_model(burden_model, val_reg_loader, reg_loss_fn, device)

    print(f'Epoch {epoch + 1} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f}')

    if val_loss < best_loss:
        torch.save(burden_model.state_dict(), 'burden_model.pt') # 다른 이름으로 저장
        best_loss = val_loss
        print(f"✅ Best Burden Model saved! (MSE: {best_loss:.4f})")

print("학습 완료!")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 과제 부담 예측 모델(Regression) 학습 시작 🚀
Epoch 1 | Train MSE: 0.0436 | Val MSE: 0.0303
✅ Best Burden Model saved! (MSE: 0.0303)
Epoch 2 | Train MSE: 0.0308 | Val MSE: 0.0238
✅ Best Burden Model saved! (MSE: 0.0238)
Epoch 3 | Train MSE: 0.0338 | Val MSE: 0.0256
Epoch 4 | Train MSE: 0.0276 | Val MSE: 0.0239
Epoch 5 | Train MSE: 0.0267 | Val MSE: 0.0237
✅ Best Burden Model saved! (MSE: 0.0237)
Epoch 6 | Train MSE: 0.0256 | Val MSE: 0.0232
✅ Best Burden Model saved! (MSE: 0.0232)
Epoch 7 | Train MSE: 0.0242 | Val MSE: 0.0231
✅ Best Burden Model saved! (MSE: 0.0231)
Epoch 8 | Train MSE: 0.0205 | Val MSE: 0.0222
✅ Best Burden Model saved! (MSE: 0.0222)
Epoch 9 | Train MSE: 0.0187 | Val MSE: 0.0218
✅ Best Burden Model saved! (MSE: 0.0218)
Epoch 10 | Train MSE: 0.0197 | Val MSE: 0.0218
학습 완료!
